In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/googlenewsvectors/GoogleNews-vectors-negative300.bin
/kaggle/input/2-recommended-reads-conversion-of-data-to-num/__results__.html
/kaggle/input/2-recommended-reads-conversion-of-data-to-num/vectorizedData.csv
/kaggle/input/2-recommended-reads-conversion-of-data-to-num/__notebook__.ipynb
/kaggle/input/2-recommended-reads-conversion-of-data-to-num/__output__.json
/kaggle/input/2-recommended-reads-conversion-of-data-to-num/custom.css
/kaggle/input/2-recommended-reads-conversion-of-data-to-num/__results___files/__results___9_0.png


In [2]:
from gensim.models import KeyedVectors
import numpy as np
import ast
import pandas as pd
from sklearn.cluster import KMeans

# Veri setini yükleme
data_path = '/kaggle/input/2-recommended-reads-conversion-of-data-to-num/vectorizedData.csv'
data = pd.read_csv(data_path)
data = data.drop_duplicates(subset=['booktitle', 'authorname'], keep='first')
data["word2vec"] = data["word2vec"].apply(lambda x: x.strip('[]'))
data["word2vec"] = data["word2vec"].apply(lambda x: x.split())
data["word2vec"] = data["word2vec"].apply(lambda x:  [float(y) for y in x])

# Google News Binary modelinin yolu
model_path = '/kaggle/input/googlenewsvectors/GoogleNews-vectors-negative300.bin'

# Google News Binary modelini yükleme
word_vectors = KeyedVectors.load_word2vec_format(model_path, binary=True)

# Kullanıcının girdiği kitap betimlemesi
user_description = 'planet travel'
user_vector = np.mean([word_vectors[word] for word in user_description.split() if word in word_vectors.key_to_index], axis=0)

X = np.array(data["word2vec"].tolist())

# KMeans algoritması kullanarak kümeleme yapma
kmeans = KMeans(n_clusters=10, random_state=0).fit(X)

# Kullanıcının girdiği kitaba benzer kitapların belirlenmesi
similar_books = []
for i, label in enumerate(kmeans.labels_):
    if label == kmeans.predict([user_vector])[0]:
        book = data.iloc[i][['booktitle', 'authorname']].values.tolist()
        similarity = np.dot(user_vector, X[i]) / (np.linalg.norm(user_vector) * np.linalg.norm(X[i]))
        similarity_percent = round(similarity * 100, 2)
        similar_books.append((book[0], book[1], similarity_percent))

# Yakınlık oranlarını hesaplama ve yüzde olarak gösterme
print("Kullanıcının girdiği kitaba benzer kitaplar:")
for book in sorted(similar_books, key=lambda x: x[2], reverse=True)[:10]:
    print(f"{book[0]} by {book[1]} ({book[2]}%)")


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Kullanıcının girdiği kitaba benzer kitaplar:
Curiosity: The Story of a Mars Rover by Markus Motum (59.72%)
Dalek Empire II: Dalek War - Chapter Three by Nicholas Briggs (58.91%)
The Three-Cornered War by John Dalmas (58.45%)
A World Out of Time by Larry Niven (56.9%)
Star Marines by Ian Douglas (56.44%)
Vortex by Robert Charles Wilson (56.22%)
The Forge by S.M. Stirling (56.14%)
The Clockwork Rocket by Greg Egan (56.0%)
The Lord of the Sands of Time by Issui Ogawa (55.56%)
On Silver Wings by Evan Currie (55.53%)
